In [ ]:
from __future__ import print_function
import time
from kubernetes import client, config
from pprint import pprint
import yaml
import argparse

class SubmitKatib(object):
    def submit(self):
                # 입력 값을 받게 추가합니다.
        parser = argparse.ArgumentParser()
        parser.add_argument('--experiment_name', required=False, type=str, default='experiment-name')        
        parser.add_argument('--yaml_name', required=False, type=str, default='random-examle.yaml')                
        args = parser.parse_args()  
        
        config.load_kube_config()
        api = client.CustomObjectsApi()

        with open(args.yaml_name) as f:
            dep = yaml.safe_load(f)
            dep['metadata']['name'] = args.experiment_name

            group = 'kubeflow.org' # str | The custom resource's group name
            version = 'v1alpha3' # str | The custom resource's version
            namespace = 'kubeflow' # str | The custom resource's namespace
            plural = "experiments"
            api_response = api.create_namespaced_custom_object(group=group, plural=plural, version=version, namespace=namespace, body=dep)
            print(api_response)
            
if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils
        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'
        fairing.config.set_builder(
            'append',
            image_name='experiement-runner',
            base_image='brightfly/kubeflow-sdk-jupyter:latest',
            registry=DOCKER_REGISTRY,
            push=True)
        
        fairing.config.set_deployer('job',
                                    namespace='dudaji',
                                    pod_spec_mutators=[
                                    k8s_utils.mounting_pvc(pvc_name="caltech101", 
                                                          pvc_mount_path="/result")]
                                    )
        # python3
        import IPython
        ipy = IPython.get_ipython()
        if ipy is None:
            fairing.config.set_preprocessor('python', input_files=[__file__])        
        fairing.config.run()
    else:
        katib = SubmitKatib()
        katib.submit()